In [ ]:
# default_exp ai

# AI - Adding player agents

This is a step towards using this as a game for RL. We define the Swoggle class, which contains a board and multiple Agents, which follow simple rules in turn. Interactive play, as well as the option to add an agent of your own.

## Board Setup

In [ ]:
# export
from swoggle.core import *
class Swoggle:
    """ The game itself"""
    
    def __init__(self, agents=[], setup='basic'):
        self.board = Board()
        self.board.reset()
        self.agents = agents
        
        # Set up basic layout
        # Add two players (with bases) and a central drone
        self.board.board[0][0].player = 1
        self.board.board[0][0].base = 1 
        self.board.board[7][0].player = 2
        self.board.board[7][0].base = 2
        self.board.board[7][7].player = 3
        self.board.board[7][7].base = 3
        self.board.board[0][7].player = 4
        self.board.board[0][7].base = 4
        self.board.board[3][2].drone = True
        self.board.board[5][3].drone = True
        self.board.board[4][5].drone = True
        self.board.board[2][4].drone = True

    def dice(self):
        return random.choice([1, 2, 3, 4, 5, 6])
    
    def move(self, player, start_loc, end_loc, dice_roll, drone=False, powerjump=False):
        """ Moves a player not under internal agent control the does the agent's moves """
        self.board.move(player, start_loc, end_loc, dice_roll, drone=False, powerjump=False)
        for a in self.agents:
            a.move(self.board, self.dice())
            
    def move_agents(self):
        for a in self.agents:
            a.move(self.board, self.dice())
            
    def show(self):
        self.board.show()
    
    def interactive_play(self):
        pass
        # TODO

In [ ]:
s = Swoggle(agents=[])
s.show()

[1.1][...][...][...][...][...][...][4.4]
[...][...][...][...][...][...][...][...]
[...][...][...][...][.d.][...][...][...]
[...][...][.d.][...][...][...][...][...]
[...][...][...][...][...][.d.][...][...]
[...][...][...][.d.][...][...][...][...]
[...][...][...][...][...][...][...][...]
[2.2][...][...][...][...][...][...][3.3]
Spa: []


In [ ]:
s.move(1, (0, 0), (3, 2), 5)
s.show()

Moved 1 (0, 0) (3, 2)
[..1][...][...][...][...][...][...][4.4]
[...][...][...][...][...][...][...][...]
[...][...][...][...][.d.][...][...][...]
[...][...][1d.][...][...][...][...][...]
[...][...][...][...][...][.d.][...][...]
[...][...][...][.d.][...][...][...][...]
[...][...][...][...][...][...][...][...]
[2.2][...][...][...][...][...][...][3.3]
Spa: []


In [ ]:
s.board.is_valid_move(1, (3, 2), (3, 3), 6, drone=True, powerjump=False)

True

In [ ]:
s.show()

[..1][...][...][...][...][...][...][4.4]
[...][...][...][...][...][...][...][...]
[...][...][...][...][.d.][...][...][...]
[...][...][1d.][...][...][...][...][...]
[...][...][...][...][...][.d.][...][...]
[...][...][...][.d.][...][...][...][...]
[...][...][...][...][...][...][...][...]
[2.2][...][...][...][...][...][...][3.3]
Spa: []


# Random Agent

In [ ]:
#export
import random
class RandomAgent:
    """ Given a swoggle board on which it is a player, make a random valid move """
    
    def __init__(self, player):
        self.player = player
    
    def move(self, board, dice_roll):
        
        # If in jail, try to escape
        if self.player in board.jail:
            board.move(self.player, (0, 0), (0, 0), dice_roll, False, False)
            return 'escape'
        
        # Get start_loc
        start_loc = (9, 9)
        for row in board.board:
            for cell in row:
                if cell.player == self.player:
                    start_loc = (cell.y, cell.x)
        if start_loc == (9, 9):
            return None
        
        # Make a random move within reach
        move = ()
        count = 0
        while True:
            count += 1
            end_x = random.choice(range(8))
            end_y = random.choice(range(8))
            drone = random.choice([True, False])
            powerjump = random.choice([True, False])
            valid = board.is_valid_move(self.player, start_loc, (end_x, end_y), dice_roll, drone=drone, powerjump=powerjump)
            move = ()
            if valid:
                move = ((self.player, start_loc, (end_x, end_y), dice_roll, drone, powerjump))
#                 print(f'{self.player} took {count} tries to guess a random move')
                break
        board.move(*move)
        return move
            

In [ ]:
RandomAgent(1).move(s.board, 6)

Moved 1 (1, 0) (2, 0)


(1, (1, 0), (2, 0), 6, False, False)

### A game with random agents

In [ ]:
sr = Swoggle([RandomAgent(i+1) for i in range(4)])
sr.show()

[1.1][...][...][...][...][...][...][4.4]
[...][...][...][...][...][...][...][...]
[...][...][...][...][.d.][...][...][...]
[...][...][.d.][...][...][...][...][...]
[...][...][...][...][...][.d.][...][...]
[...][...][...][.d.][...][...][...][...]
[...][...][...][...][...][...][...][...]
[2.2][...][...][...][...][...][...][3.3]
Spa: []


In [ ]:
sr.move_agents()
sr.show()

Moved 1 (0, 0) (3, 5)
Moved 2 (7, 0) (6, 0)
Moved 3 (7, 7) (4, 7)
Moved 4 (0, 7) (2, 3)
[..1][...][...][...][...][...][...][..4]
[...][...][...][...][...][...][...][...]
[...][...][...][4..][.d.][...][...][...]
[...][...][.d.][...][...][1..][...][...]
[...][...][...][...][...][.d.][...][3..]
[...][...][...][.d.][...][...][...][...]
[2..][...][...][...][...][...][...][...]
[..2][...][...][...][...][...][...][..3]
Spa: []


# Basic Player

In [ ]:
class BasicAgent:
    """ Given a swoggle board on which it is a player, make a sensible move """
    
    def __init__(self, player):
        self.player = player
    
    def move(self, board, dice_roll):
        
        # If in jail, try to escape
        if self.player in board.jail:
            board.move(self.player, (0, 0), (0, 0), dice_roll, False, False)
            return 'escape'
        
        # Get start_loc
        start_loc = (9, 9)
        for row in board.board:
            for cell in row:
                if cell.player == self.player:
                    start_loc = (cell.y, cell.x)
        if start_loc == (9, 9):
            return None
        
        
        # If bases in range, take them
        
        for row in board.board:
            for cell in row:
                if cell.player == None and cell.base != None and cell.base != self.player: # Normal move
                    move = (self.player, start_loc, (cell.x, cell.y), dice_roll, False, False)
                    if board.is_valid_move(*move):
                        board.move(*move)
                        return (move)
                    
                if cell.base != None and cell.base != self.player: # Drone attack
                    move = (self.player, start_loc, (cell.x, cell.y), dice_roll, True, False)
                    if board.is_valid_move(*move):
                        board.move(*move)
                        return (move)
        
        # If on base and player in range, take or powerjump them
        if board.board[start_loc[0]][start_loc[1]].base == self.player:
            for row in board.board:
                for cell in row:
                    if cell.player != None and cell.player != self.player:
                        # try normal move
                        move = (self.player, start_loc, (cell.x, cell.y), dice_roll, False, False)
                        if board.is_valid_move(*move):
                            board.move(*move)
                            return (move)
                        # Try powerjump
                        move = (self.player, start_loc, (cell.x, cell.y), dice_roll, False, True)
                        if board.is_valid_move(*move):
                            board.move(*move)
                            return (move)
        
        # If players in range and takeable, take them
        for row in board.board:
            for cell in row:
                if cell.player != None and cell.player != self.player:
                    # Normal take
                    move = (self.player, start_loc, (cell.x, cell.y), dice_roll, False, False)
                    if board.is_valid_move(*move):
                        board.move(*move)
                        return (move)
                    # Drone take
                    move = (self.player, start_loc, (cell.x, cell.y), dice_roll, True, False)
                    if board.is_valid_move(*move):
                        board.move(*move)
                        return (move)
        
        # TODO: If player close to your base and base reacheable, go back to base
        
        # Else move randomly
        
        # Make a random move within reach
        move = ()
        count = 0
        while True:
            count += 1
            end_x = random.choice(range(8))
            end_y = random.choice(range(8))
            drone = random.choice([True, False])
            powerjump = random.choice([True, False])
            valid = board.is_valid_move(self.player, start_loc, (end_x, end_y), dice_roll, drone=drone, powerjump=powerjump)
            move = ()
            if valid:
                move = ((self.player, start_loc, (end_x, end_y), dice_roll, drone, powerjump))
#                 print(f'{self.player} took {count} tries to guess a random move')
                break
        board.move(*move)
        return move

### Here's a way to get win rates for different agents over n games:

In [ ]:
from IPython.display import clear_output

def win_rates(n, agents):
    wins = {}
    for i in range(n):
        rounds = 0
        sr = Swoggle(agents)
        while True:
            sr.move_agents()
            rounds += 1
            players = []
            for row in sr.board.board:
                for cell in row:
                    if cell.player != None:
                        players.append(cell.player)
            if len(players) <= 1:
                clear_output()
                print("Winner:", players, rounds)
                if len(players) == 1:
                    if players[0] in wins:
                        wins[players[0]] += 1
                    else:
                        wins[players[0]] = 1
                break
    return wins


print(win_rates(500, [RandomAgent(i+1) for i in range(3)]+[BasicAgent(4)]))

Winner: [4] 47
{3: 52, 4: 311, 2: 128, 1: 9}


Interestingly, the one opposite the non-random player also gets some advantage.